<a href="https://colab.research.google.com/github/jantuitman/deeplearning/blob/main/training_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import T5ForConditionalGeneration, Adafactor, T5Tokenizer, AutoTokenizer, PreTrainedModel
import torch
import time

import json

In [3]:
# read a json file containing an array of objects
# each object has a "question" and "answer" field
# the question is the input and the answer is the output
# we will return a list of tuples
def read_dataset(filename):
    with open(filename) as f:
        data = json.load(f)
        # data has the following structure:
        # data = {
        #     "paragraphs": [
        #         {
        #             "translated_context": "text",
        #             "translated_question": "text",
        #             "translated_answers": [
        #                 "text",
        #                 "text",
        #                 "text"
        #             ],
        #             "is_impossible": true
        #         },
        #         ...
        #     ]
        # }
        # if is_impossible is true, then translated_answers is empty and we want to return the answer "Onbekend"
        # if is_impossible is false, then translated_answers is not empty and we want to return the first answer
        # we will return a list of tuples
        # we need to make a question_text from the translated_context and translated_question
        result= []
        for paragraph in data['paragraphs']:
            question_text = f"Context: {paragraph['translated_context']}\n\nVraag: {paragraph['translated_question']}"
            answer_text = "Onbekend"
            if not paragraph['is_impossible']:
               answer_text = paragraph['translated_answers'][0]
            result.append((question_text, answer_text))
        return result

In [3]:
def tokenize_dataset(dataset, tokenizer: T5Tokenizer):
    result = []
    for (input, output) in dataset:
        print(input)
        print(output)
        # Tokenize the input and output
        # we will print a warning if the input or output exceeds 512 tokens
        input_ids = tokenizer.encode(input, return_tensors='pt')
        target_ids = tokenizer.encode(output, return_tensors='pt')
        if input_ids.shape[1] > 512:
            print(f'Input exceeds 512 tokens: {input}')
            input_ids = input_ids[:, :512]
        if target_ids.shape[1] > 512:
            print(f'Output exceeds 512 tokens: {output}')
            # restrict to the first 512
            target_ids = target_ids[:, :512]
        result.append((input_ids, target_ids))
    return result


In [4]:
def get_best_device(allow_mps=False):
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.has_mps and allow_mps:
        return torch.device('mps')
    else:
        return torch.device('cpu')

In [16]:
def train(training_data, model_path, session_name, model, start_epoch=None):
    torch.cuda.empty_cache()
    start = 0
    if start_epoch is not None:
        start = start_epoch
        if model is None:
            model = T5ForConditionalGeneration.from_pretrained(f"models/t5/{session_name}/epoch_{start_epoch}")

    # Load the model, tokenizer, and optimizer
    optimizer = Adafactor(model.parameters())

    # Set the device to run on (CPU or GPU)
    device = get_best_device()
    model.to(device)

    # Set the number of epochs to train for

    num_epochs = 60
    batch_size = 16

    start_time = time.time()
    total_num_tokens = 0
    min_loss = 1000
    total_loss = 0
    model.train()
    for epoch in range(start, num_epochs):
        # Set the model to training mode

        # Iterate through the training data in batches of size batch_size
        for i in range(0, len(training_data), batch_size):
            if i % 512 ==0:
              elapsed_time = time.time() - start_time
              print(f"epoch {epoch} | record {i} of {len(training_data)} | Elapsed time: {elapsed_time:.2f}")
            # loss that was in this batch
            total_loss = 0
            # Zero out the gradients from the previous iteration
            optimizer.zero_grad()

            # Get the current batch of data
            batch = training_data[i:i + batch_size]

            # Tokenize the input and target sequences for the batch
            for input_ids, target_ids in batch:
                # Concatenate the input and target sequences for the batch
                # update total number of tokens
                num_tokens = input_ids.numel() + target_ids.numel()
                total_num_tokens += num_tokens
                # Forward pass
                input_ids = input_ids.to(device)
                target_ids = target_ids.to(device)
                outputs = model(input_ids=input_ids, labels=target_ids)
                loss = outputs[0]
                # Accumulate the loss
                total_loss += loss
                del input_ids
                del target_ids
                torch.cuda.empty_cache()

            # Backward pass
            total_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

        elapsed_time = time.time() - start_time
        ett = total_num_tokens / elapsed_time / 1000  # elapsed time per 1K tokens
        # Print the loss for the epoch
        print(
            f'Epoch {epoch + 1} | Loss: {total_loss.item():.6f} | Elapsed time: {elapsed_time:.2f} | Elapsed time per 1K token: {ett:.2f} seconds | Epochs completed {epoch / num_epochs * 100 :.1f}%')
        if (total_loss.item() < min_loss) or (epoch % 10 == 0):
            min_loss = total_loss.item()
            model.save_pretrained(f'{model_path}/{session_name}/epoch_{epoch + 1}')

    print("Saving final version")
    model.save_pretrained(f'{model_path}/{session_name}/epoch_{epoch + 1}')
    #return model


In [7]:
model_name = 'yhavinga/t5-base-dutch'
model: PreTrainedModel = T5ForConditionalGeneration.from_pretrained(model_name)
session_name = 'squad_on_basemodel'
model_path = 'models/t5'
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_name)
device = get_best_device()

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def test(context, question):
  model.to(device)
  with torch.no_grad():
   
    input_text = f'Context:{context}\n\nVraag: {question}'
    print(input_text)
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    input_ids.to(device)
    #print(input_ids)
    # Generate the output sequence
    generated_ids = model.generate(input_ids, do_sample=True,
        max_length=200,
        top_k=0,
        temperature=0.7
    )

    output_text = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)
    print("ANTWOORD:",output_text)
    del input_ids
    del generated_ids
    torch.cuda.empty_cache()


In [9]:
context = "Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag"
print("HET MODEL IS NOG NIET GETRAIND.DIT ZIJN DE ANTWOORDEN MOMENTEEL:")
test(context, "Wie is de koning van Nederland?")
test(context,"Met wie is Willem Alexander getrouwd?")
test(context,"Hoe veel kinderen heeft Willem Alexander?")
test(context,"Hoe heten de kinderen van Willem Alexander en Maxima?" )
test(context,"Waar wonen Willem Alexander en Maxima?")

HET MODEL IS NOG NIET GETRAIND.DIT ZIJN DE ANTWOORDEN MOMENTEEL:
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Wie is de koning van Nederland?
ANTWOORD: , Maxima, Maxima en Maxima. is getrouwd met Maxima. Ze hebben 2 kinderen, Maxima en Maxima. Willem Alexander en Maxima. Willem Alexander en Maxima. nu koning Willem Alexander. Willem Alexander. Koning  "
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Met wie is Willem Alexander getrouwd?
ANTWOORD: ). Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Hij is getrouwd met Maxima. Hij is getrouwd met Maxima. Hij is getrouwd met Maxima. Zij hebben twee kinderen, Maxima en Maxima. en heeft drie kinderen. Willem Maxim... 1 vraag: Met wie? Den haag 

In [10]:
dataset = read_dataset('dev-nl.json')
# tokenize the dataset
tokenized_dataset = tokenize_dataset(dataset, tokenizer)


Streaminguitvoer ingekort tot de laatste 5000 regels.

Vraag: Welke onderdelen van een conventionele zuigerstoommachine zouden kunnen worden vervangen door een zuigerloze rotatiemotor?
cilinders en ventielen
Context: Het is mogelijk om in plaats van de cilinders en kleppen van een conventionele heen-en-weergaande stoommachine een mechanisme te gebruiken dat gebaseerd is op een roterende motor zonder zuiger, zoals de Wankelmotor. Er zijn veel van dergelijke motoren ontworpen, vanaf de tijd van James Watt tot heden, maar er zijn er betrekkelijk weinig daadwerkelijk gebouwd en nog minder in serieproductie gegaan; zie de link onderaan het artikel voor meer details. Het grootste probleem is de moeilijkheid om de rotoren stoomdicht te maken tegen slijtage en thermische uitzetting; de resulterende lekkage maakte ze zeer inefficiënt. Het gebrek aan expansieve werking, of enig middel om de afsluiting te controleren is ook een ernstig probleem bij veel van dergelijke ontwerpen.[citation needed].

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaminguitvoer ingekort tot de laatste 5000 regels.
Vraag: Wat is de naam van het agentschap voor de beveiliging van de buitengrenzen?
Onbekend
Context: De eerste historische verwijzing naar Warschau dateert uit het jaar 1313, toen Kraków fungeerde als Poolse hoofdstad. Door de centrale ligging tussen de hoofdsteden Kraków en Vilnius van het Pools-Litouwse Gemenebest werd Warschau de hoofdstad van het Gemenebest en van de kroon van het Koninkrijk Polen toen koning Sigismund III Vasa in 1596 zijn hofhouding van Kraków naar Warschau verplaatste. Na de Derde Verdeling van Polen in 1795 werd Warschau opgenomen in het Koninkrijk Pruisen. In 1806, tijdens de Napoleontische oorlogen, werd de stad de officiële hoofdstad van het Groothertogdom Warschau, een marionettenstaat van het Eerste Franse Rijk, opgericht door Napoleon Bonaparte. In overeenstemming met de besluiten van het Congres van Wenen annexeerde het Russische Rijk Warschau in 1815 en werd het onderdeel van het "Congresrijk". Pas i

In [11]:
train(tokenized_dataset, model_path, session_name, model)

epoch 0 | record 0 of 11873 | Elapsed time: 0.00
epoch 0 | record 512 of 11873 | Elapsed time: 64.28
epoch 0 | record 1024 of 11873 | Elapsed time: 128.00
epoch 0 | record 1536 of 11873 | Elapsed time: 193.59
epoch 0 | record 2048 of 11873 | Elapsed time: 259.56
epoch 0 | record 2560 of 11873 | Elapsed time: 326.39
epoch 0 | record 3072 of 11873 | Elapsed time: 391.70
epoch 0 | record 3584 of 11873 | Elapsed time: 458.58
epoch 0 | record 4096 of 11873 | Elapsed time: 524.62
epoch 0 | record 4608 of 11873 | Elapsed time: 590.91
epoch 0 | record 5120 of 11873 | Elapsed time: 657.81
epoch 0 | record 5632 of 11873 | Elapsed time: 724.47
epoch 0 | record 6144 of 11873 | Elapsed time: 790.74
epoch 0 | record 6656 of 11873 | Elapsed time: 858.89
epoch 0 | record 7168 of 11873 | Elapsed time: 926.40
epoch 0 | record 7680 of 11873 | Elapsed time: 994.99
epoch 0 | record 8192 of 11873 | Elapsed time: 1063.72
epoch 0 | record 8704 of 11873 | Elapsed time: 1131.28
epoch 0 | record 9216 of 11873 | 

In [12]:
context = "Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag"
print("NA 4 epochs:")
test(context, "Wie is de koning van Nederland?")
test(context,"Met wie is Willem Alexander getrouwd?")
test(context,"Hoe veel kinderen heeft Willem Alexander?")
test(context,"Hoe heten de kinderen van Willem Alexander en Maxima?" )
test(context,"Waar wonen Willem Alexander en Maxima?")

NA 4 epochs:
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Wie is de koning van Nederland?
ANTWOORD: Onbekend
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Met wie is Willem Alexander getrouwd?
ANTWOORD: Onbekend
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Hoe veel kinderen heeft Willem Alexander?
ANTWOORD: 3 kinderen
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Hoe heten de kinderen van Willem Alexander en Maxima?
ANTWOORD: Onbekend
Con

In [13]:
model_path = 'drive/MyDrive/OutputModels/t5'

In [ ]:
train(tokenized_dataset, model_path, session_name, model,4)

epoch 4 | record 0 of 11873 | Elapsed time: 0.00
epoch 4 | record 512 of 11873 | Elapsed time: 65.35
epoch 4 | record 1024 of 11873 | Elapsed time: 130.23
epoch 4 | record 1536 of 11873 | Elapsed time: 196.63
epoch 4 | record 2048 of 11873 | Elapsed time: 263.05
epoch 4 | record 2560 of 11873 | Elapsed time: 330.86
epoch 4 | record 3072 of 11873 | Elapsed time: 396.50
epoch 4 | record 3584 of 11873 | Elapsed time: 463.80
epoch 4 | record 4096 of 11873 | Elapsed time: 530.32
epoch 4 | record 4608 of 11873 | Elapsed time: 596.70
epoch 4 | record 5120 of 11873 | Elapsed time: 664.20
epoch 4 | record 5632 of 11873 | Elapsed time: 731.18
epoch 4 | record 6144 of 11873 | Elapsed time: 797.70
epoch 4 | record 6656 of 11873 | Elapsed time: 866.01
epoch 4 | record 7168 of 11873 | Elapsed time: 933.28
epoch 4 | record 7680 of 11873 | Elapsed time: 1002.02
epoch 4 | record 8192 of 11873 | Elapsed time: 1069.98
epoch 4 | record 8704 of 11873 | Elapsed time: 1136.83
epoch 4 | record 9216 of 11873 |

In [12]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('drive/MyDrive/OutputModels/t5/squad_on_basemodel/epoch_17')
tokenizer = tokenizer = tokenizer = AutoTokenizer.from_pretrained('yhavinga/t5-base-dutch')
device = get_best_device()

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
context = "Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag"
print("NA 17 epochs:")
test(context, "Wie is de koning van Nederland?")
test(context,"Met wie is Willem Alexander getrouwd?")
test(context,"Hoe veel kinderen heeft Willem Alexander?")
test(context,"Hoe heten de kinderen van Willem Alexander en Maxima?" )
test(context,"Waar wonen Willem Alexander en Maxima?")

NA 17 epochs:
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Wie is de koning van Nederland?
ANTWOORD: Onbekend
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Met wie is Willem Alexander getrouwd?
ANTWOORD: Onbekend
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Hoe veel kinderen heeft Willem Alexander?
ANTWOORD: Onbekend
Context:Willem Alexander is de koning van Nederland. Hij is getrouwd met Maxima. Ze hebben 3 kinderen, Amalia, Alexia en Alexander. Willem Alexander en Maxima wonen in Den haag

Vraag: Hoe heten de kinderen van Willem Alexander en Maxima?
ANTWOORD: 3 kinderen, A